In [18]:
PIPELINE_NAME='basic_pipeline_2'

In [22]:
%%writefile pipeline.py

import kfp

from typing import NamedTuple

from kfp.v2.dsl import pipeline
from kfp.v2.dsl import component
from kfp.v2 import compiler

@component
def concat(a: str, b: str) -> str:
    return a+b

@component
def reverse(a: str) -> NamedTuple("outputs", [("before", str), ("after", str)]):
    return a, a[::-1]

@pipeline(name="basic-pipeline", description="A simple intro pipeline",
          pipeline_root='gs://tron-302502-bucket/basic-pipe')
def basic_pipeline(a: str='stres', b: str='sed'):
    concat_task = concat(a,b)
    reverse_task = reverse(concat_task.output)
    
if __name__ == '__main__':
    compiler.Compiler().compile(
        pipeline_func=basic_pipeline, package_path="basic_pipeline_2.json"
    )

Overwriting pipeline.py


In [23]:
%%writefile requirements.txt

kfp==1.7.2

Overwriting requirements.txt


In [24]:
%%writefile cloudbuild.yaml

steps:
    - name: python
      entrypoint: pip
      args: ["install", "-r", "requirements.txt", "--user"]
    
    - name: 'python'
      entrypoint: 'python'
      args: ['pipeline.py']
      id: 'compile'
    
    - name: 'gcr.io/cloud-builders/gsutil'
      args: ['cp', 'basic_pipeline_2.json', 'gs://tron-302502-bucket/aut']
      id: 'upload'
      waitFor: ['compile']

Overwriting cloudbuild.yaml


In [25]:
!gcloud builds submit .

Creating temporary tarball archive of 13 file(s) totalling 13.5 KiB before compression.
Uploading tarball of [.] to [gs://tron-302502_cloudbuild/source/1637762899.953049-d450cdf2eb53400b8fb2a283893e369c.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/tron-302502/locations/global/builds/fd76aa74-e212-426d-b265-1cd059c7dc81].
Logs are available at [https://console.cloud.google.com/cloud-build/builds/fd76aa74-e212-426d-b265-1cd059c7dc81?project=621054987025].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "fd76aa74-e212-426d-b265-1cd059c7dc81"

FETCHSOURCE
Fetching storage object: gs://tron-302502_cloudbuild/source/1637762899.953049-d450cdf2eb53400b8fb2a283893e369c.tgz#1637762900551946
Copying gs://tron-302502_cloudbuild/source/1637762899.953049-d450cdf2eb53400b8fb2a283893e369c.tgz#1637762900551946...
/ [1 files][  2.8 KiB/  2.8 KiB]                                                
Operation completed over 1 objects/2.8 KiB.     

In [ ]:
%%bash

 gcloud scheduler jobs create http \
    vertex-ai-basic-pipeline \
    --schedule "0 */12 * * *" \
    --uri https://vertex-ai-pipeline-schedule-mqdfut5tfa-uc.a.run.app \
    --oidc-service-account-email workbench@tron-302502.iam.gserviceaccount.com \
    --message-body "{
    \"templatePath\": \"gs://tron-302502-bucket/aut/basic_pipeline.json\",
    \"project\": \"tron-302502\",
    \"location\": \"us-central1\"
}"